# Calculating Equity of Access to Opportunities

In this notebook, we will estimate the *change* in the level of access to opportunities (in the form of employment opportunities) accross our study region by comparing three different potential transit scenarios with a baseline "business as usual" scenario.

The basic workflow for this process is as follows:

1. Load travel time and demand matrices for each scenario
2. Estimate or specify an impedance function for the measure of access to opportunities. In this workbook we will use an impedance function based on observed model behaviour.
3. Calculate the level of access to employment for each zone in our study area, and calculate this difference from the business as usual function.
4. Create population weighted summaries for various population groups to compare the impact of each scenario on these groups, and visualize these results.

To get started, lets load our required Python modules and specify the location of various datasets we will need, including the matrices folder containing the travel time and demand matrices, the opportunities folder with the employment specifciation, and the folder containing the link created between dissemination areas (census zones) and transport analysis zones from the model.

In [1]:
import os
import pandas as pd
import altair as alt
import numpy as np
from scipy.stats import norm

# Replace with your data folder paths
mx_folder = r"data/matrices/input"
counts_folder = r"data/counts/input"
link_folder = r"data/counts/interim"
output_folder = r"data/counts/output"

Next, we load in our four travel time matrices. Since these can be quite large, it's useful to specify specific data types for each column to make loading the data a bit less memory intensive. We do that using the `dtype` keyword.

In [2]:
dtypes = {'i':int, 'j':int, 'travel_time':float, 'demand':float}
mx_BAU = pd.read_csv(os.path.join(mx_folder, 'bau_times_flows.csv'), dtype=dtypes)
mx_A = pd.read_csv(os.path.join(mx_folder, 'scenario_A_times_flows.csv'), dtype=dtypes)
mx_B = pd.read_csv(os.path.join(mx_folder, 'scenario_B_times_flows.csv'), dtype=dtypes)
mx_C = pd.read_csv(os.path.join(mx_folder, 'scenario_C_times_flows.csv'), dtype=dtypes)

## Estimating an Admittance Function

Now that we have travel times and demand, we need to estimate or provide a defined impedance function to weight the travel times against to get a more useful measure of access to opportunities. 

In the code below, we are going to use an approach which uses the overall characteristics of the distribution of trip lenghts in our dataset to generate a smooth curve. This requires the following steps:

1. Calculate the mean and standard deviation of the travel times accross all origin-destination points, weighted by the demand (number of trips taken)
2. Use this mean and standard deviation to generate a cumulative normal distribution curve, and use 1 minus this curve as the impedance function

Let's start by generating the demand-weighted mean and standard deviation of time in the matrices. Here we will use the business as usual scenario as our baseline for all admittance/impedance functions.

In [3]:
mean = np.average(mx_BAU.travel_time, weights=mx_BAU.demand)
std = np.sqrt(np.average((mx_BAU.travel_time - mean)**2, weights=mx_BAU.demand))
print(mean, std)

47.40209771303543 19.996002157469057


Our impedance function is now easily specified as `1 - norm.cdf(<travel_time>, loc=mean, scale=std)`. To get a sense of how well our normal curve fits the actual distribution, we'll generate a histogram of our PDF and plot that alongside our theoretical curve.

In [4]:
# First we generate a histogram as our PDF
hist_vals, hist_bins=np.histogram(mx_BAU.travel_time, 200, weights=mx_BAU.demand, density=False)
# Normalize them between 0 and 1
hist_vals /= np.sum(hist_vals)
imp = pd.DataFrame({'pdf': hist_vals, 'cdf':  np.cumsum(hist_vals), 'imp':  1-np.cumsum(hist_vals), 'travel_time': hist_bins[1:]})
# Create a theoretical impedence
imp['norm'] = 1-norm.cdf(imp['travel_time'], loc=mean, scale=std)

# Plot the data - 
real = alt.Chart(imp).mark_line(clip=True, color="#0D0A45", size=2).encode(
    alt.X('travel_time:Q', title="Travel time (min)", scale=alt.Scale(domain=(1, 180))),
    alt.Y('imp:Q', title="Admittance")
)

# Plot the model - dashed line
model = alt.Chart(imp).mark_line(color='#6D0D4A', clip=True, strokeDash=[5], size=2).encode(
    alt.X('travel_time:Q'),
    alt.Y('norm:Q')
)

(real+model).properties(
    width = 300,
    height=200
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0)

alt.LayerChart(...)

With this function estimated, we can now apply it to the travel time for each origin-destination pair:

In [5]:
mx_BAU['impedance'] = 1-norm.cdf(mx_BAU.travel_time, loc=mean, scale=std)
mx_A['impedance'] = 1-norm.cdf(mx_A.travel_time, loc=mean, scale=std)
mx_B['impedance'] = 1-norm.cdf(mx_B.travel_time, loc=mean, scale=std)
mx_C['impedance'] = 1-norm.cdf(mx_C.travel_time, loc=mean, scale=std)

## Calculating Access to Opportunities
The final ingredient we need is the actual opportunities to access. For this workbook, we are considering projected employment numbers, however any set of opportunities will do. We do this with the following steps for each scenario matrix:

1. Load the opportunities, and merge them with the matrix
2. Multiply the calculated admittance/impedance by the number of opportunities
3. Group and sum for each origin `i`

In [6]:
ops = pd.read_csv(os.path.join(counts_folder, 'taz_employment.csv'))

acc_BAU = pd.merge(mx_BAU, ops, left_on='i', right_on='taz_id')
acc_BAU['emp_imp_BAU'] = acc_BAU.impedance * acc_BAU.employment
access_BAU = acc_BAU[['i', 'emp_imp_BAU']].groupby('i', as_index=False).sum()

acc_A = pd.merge(mx_A, ops, left_on='i', right_on='taz_id')
acc_A['emp_imp_A'] = acc_A.impedance * acc_A.employment
access_A = acc_A[['i', 'emp_imp_A']].groupby('i', as_index=False).sum()

acc_B = pd.merge(mx_B, ops, left_on='i', right_on='taz_id')
acc_B['emp_imp_B'] = acc_B.impedance * acc_B.employment
access_B = acc_B[['i', 'emp_imp_B']].groupby('i', as_index=False).sum()

acc_C = pd.merge(mx_C, ops, left_on='i', right_on='taz_id')
acc_C['emp_imp_C'] = acc_C.impedance * acc_C.employment
access_C = acc_C[['i', 'emp_imp_C']].groupby('i', as_index=False).sum()

Since we are interested in the *change* in access from the original scenario, let's calcualte the percent change for each of our three intervention scenarios compared to the business as usual scenario. To do this we must merge the datasets together and calculate various percent differences. We'll also write the output so we can generate a map of the percent change spatially if we'd like.

In [7]:
delta = pd.merge(access_BAU, access_A, on='i')
delta = pd.merge(delta, access_B, on='i')
delta = pd.merge(delta, access_C, on='i')
delta['delta_A'] = 100 * (delta['emp_imp_A'] - delta['emp_imp_BAU'])/(delta['emp_imp_BAU'])
delta['delta_B'] = 100 * (delta['emp_imp_B'] - delta['emp_imp_BAU'])/(delta['emp_imp_BAU'])
delta['delta_C'] = 100 * (delta['emp_imp_C'] - delta['emp_imp_BAU'])/(delta['emp_imp_BAU'])
delta.to_csv(os.path.join(output_folder, 'access_change_by_taz.csv'), index=False)

## Equity of Access to Opportunities
The final step is to find some distributive summaries accross population groups. Using the percent change in access we calculated for our scenarios, we can do weighted summaries for population groups. To do this we:
1. Load and join the link file between TAZ and dissemination area generated by areal apportionment.
2. Load and join the demographic counts by dissemination area
3. Generate fractional counts of each demographic group in each TAZ
4. Conduct a population weighted sum for each taz by each group of interest

Let's do the first two steps together

In [8]:
taz_da = pd.read_csv(os.path.join(link_folder, 'taz_da_link.csv'), dtype={'DAUID': int, 'taz_id': int, 'frac_da_in_taz': float})
da_demo = pd.read_csv(os.path.join(counts_folder, 'da_census_profile.csv'))
taz_da_acc = pd.merge(taz_da, delta, left_on='taz_id', right_on='i')
taz_da_acc = pd.merge(taz_da_acc, da_demo, on='DAUID')
taz_da_acc.head()

,DAUID,taz_id,frac_da_in_taz,i,emp_imp_BAU,emp_imp_A,emp_imp_B,emp_imp_C,delta_A,delta_B,...,pop_2016,labour_unemployed,labour_total,income_lico,income_lim,income_total,vm_total,vm_minority,im_total,im_2011_2016
0,35190654,600,0.000153,600,220331.929808,302395.986480,249857.067138,206184.031191,37.245649,13.400299,...,748.0,35.0,625.0,115.0,120.0,750.0,690.0,670.0,690.0,45.0
1,35190657,600,0.000223,600,220331.929808,302395.986480,249857.067138,206184.031191,37.245649,13.400299,...,663.0,10.0,580.0,135.0,135.0,665.0,725.0,730.0,725.0,90.0
2,35190648,600,0.001409,600,220331.929808,302395.986480,249857.067138,206184.031191,37.245649,13.400299,...,1258.0,50.0,1030.0,290.0,345.0,1260.0,1280.0,1240.0,1285.0,160.0
3,35190648,650,0.000066,650,357890.899033,477942.854165,372461.614314,351633.757262,33.544288,4.071273,...,1258.0,50.0,1030.0,290.0,345.0,1260.0,1280.0,1240.0,1285.0,160.0
4,35190651,600,0.000306,600,220331.929808,302395.986480,249857.067138,206184.031191,37.245649,13.400299,...,457.0,20.0,405.0,110.0,100.0,460.0,435.0,420.0,440.0,75.0


Next, we use a bit of Pandas wizardry to multiply the fraction of a given da population in a TAZ by each demographic count, rename the columns for consistency, and then put them together with our original dataset to get a table containing our access changes, and the count of each demographic group in each intersected area. We will perform the calculation for three population groups: Total population, visible minorities, and low income individuals.

In [9]:
# Minus 1-11 columns
da_acc = taz_da_acc[taz_da_acc.columns[-11:-1]].multiply(taz_da_acc['frac_da_in_taz'], axis="index")
da_acc.columns = [f"f_{c}" for c in da_acc.columns]
da_acc = pd.concat([taz_da_acc[['delta_A', 'delta_B', 'delta_C']], da_acc], axis=1)
display(da_acc.head())
da_acc['A_pop_2016'] = (da_acc['f_pop_2016']/da_acc['f_pop_2016'].sum()) * da_acc['delta_A']
da_acc['B_pop_2016'] = (da_acc['f_pop_2016']/da_acc['f_pop_2016'].sum()) * da_acc['delta_B']
da_acc['C_pop_2016'] = (da_acc['f_pop_2016']/da_acc['f_pop_2016'].sum()) * da_acc['delta_C']

da_acc['A_vm_minority'] = (da_acc['f_vm_minority']/da_acc['f_vm_minority'].sum()) * da_acc['delta_A']
da_acc['B_vm_minority'] = (da_acc['f_vm_minority']/da_acc['f_vm_minority'].sum()) * da_acc['delta_B']
da_acc['C_vm_minority'] = (da_acc['f_vm_minority']/da_acc['f_vm_minority'].sum()) * da_acc['delta_C']

da_acc['A_income_lim'] = (da_acc['f_income_lim']/da_acc['f_income_lim'].sum()) * da_acc['delta_A']
da_acc['B_income_lim'] = (da_acc['f_income_lim']/da_acc['f_income_lim'].sum()) * da_acc['delta_B']
da_acc['C_income_lim'] = (da_acc['f_income_lim']/da_acc['f_income_lim'].sum()) * da_acc['delta_C']
display(da_acc.head())



,delta_A,delta_B,delta_C,f_delta_C,f_pop_2016,f_labour_unemployed,f_labour_total,f_income_lico,f_income_lim,f_income_total,f_vm_total,f_vm_minority,f_im_total
0,37.245649,13.400299,-6.421175,-0.000985,0.114742,0.005369,0.095874,0.017641,0.018408,0.115049,0.105845,0.102777,0.105845
1,37.245649,13.400299,-6.421175,-0.001431,0.147723,0.002228,0.129230,0.030079,0.030079,0.148168,0.161537,0.162651,0.161537
2,37.245649,13.400299,-6.421175,-0.009049,1.772734,0.070458,1.451444,0.408659,0.486163,1.775552,1.803736,1.747369,1.810782
3,33.544288,4.071273,-1.748338,-0.000115,0.082790,0.003291,0.067785,0.019085,0.022705,0.082922,0.084238,0.081606,0.084567
4,37.245649,13.400299,-6.421175,-0.001963,0.139697,0.006114,0.123802,0.033625,0.030568,0.140615,0.132972,0.128387,0.134501


,delta_A,delta_B,delta_C,f_delta_C,f_pop_2016,f_labour_unemployed,f_labour_total,f_income_lico,f_income_lim,f_income_total,...,f_im_total,A_pop_2016,B_pop_2016,C_pop_2016,A_vm_minority,B_vm_minority,C_vm_minority,A_income_lim,B_income_lim,C_income_lim
0,37.245649,13.400299,-6.421175,-0.000985,0.114742,0.005369,0.095874,0.017641,0.018408,0.115049,...,0.105845,0.000002,5.631274e-07,-2.698402e-07,0.000003,9.939564e-07,-4.762855e-07,0.000001,4.554150e-07,-2.182264e-07
1,37.245649,13.400299,-6.421175,-0.001431,0.147723,0.002228,0.129230,0.030079,0.030079,0.148168,...,0.161537,0.000002,7.249912e-07,-3.474023e-07,0.000004,1.573003e-06,-7.537538e-07,0.000002,7.441730e-07,-3.565939e-07
2,37.245649,13.400299,-6.421175,-0.009049,1.772734,0.070458,1.451444,0.408659,0.486163,1.775552,...,1.810782,0.000024,8.700189e-06,-4.168969e-06,0.000047,1.689885e-05,-8.097617e-06,0.000033,1.202785e-05,-5.763524e-06
3,33.544288,4.071273,-1.748338,-0.000115,0.082790,0.003291,0.067785,0.019085,0.022705,0.082922,...,0.084567,0.000001,1.234466e-07,-5.301203e-08,0.000002,2.397772e-07,-1.029682e-07,0.000001,1.706627e-07,-7.328817e-08
4,37.245649,13.400299,-6.421175,-0.001963,0.139697,0.006114,0.123802,0.033625,0.030568,0.140615,...,0.134501,0.000002,6.856046e-07,-3.285290e-07,0.000003,1.241636e-06,-5.949689e-07,0.000002,7.562728e-07,-3.623919e-07


Finally, we want to produce a visualization that shows how these scenarios differ for various population groups. To do this, we will keep only the data we are interested in, change the table format by using the `melt()` function, and plot it in Altair. We do a bit of hacky string manipulation here to get our labels correct for plotting.

In [10]:
to_plot = da_acc[['A_pop_2016', 'B_pop_2016', 'C_pop_2016', 'A_vm_minority', 'B_vm_minority', 'C_vm_minority', 'A_income_lim', 'B_income_lim', 'C_income_lim']]
pretty_names = {'income_lim': "Low Income (LIM)", 'pop_2016': 'Total Population', 'vm_minority': "Visible Minority"}
melted = to_plot.melt().groupby('variable', as_index=False).sum()
melted['scenario'] = melted.variable.str[0]
melted['demographic'] = melted.variable.str[2:]
melted['demo_name'] = melted.demographic.map(pretty_names)
melted

,variable,value,scenario,demographic,demo_name
0,A_income_lim,4.783535,A,income_lim,Low Income (LIM)
1,A_pop_2016,5.067423,A,pop_2016,Total Population
2,A_vm_minority,5.058215,A,vm_minority,Visible Minority
3,B_income_lim,1.257105,B,income_lim,Low Income (LIM)
4,B_pop_2016,1.303865,B,pop_2016,Total Population
5,B_vm_minority,1.427638,B,vm_minority,Visible Minority
6,C_income_lim,0.034852,C,income_lim,Low Income (LIM)
7,C_pop_2016,0.025408,C,pop_2016,Total Population
8,C_vm_minority,0.014673,C,vm_minority,Visible Minority


Now we assemble a plot showing the comparative differences.

In [11]:
bar = alt.Chart(melted).mark_bar().encode(
    alt.Y('scenario:N', title=None),
    alt.X('value:Q', title='Percent Change in Access'),
    alt.Color('scenario:N', title='Scenario'),
    alt.Row('demo_name:N', title='', sort=['Total Population'], spacing=35)
)

(bar).properties(
    title="Change in Access to Opportunities for SmartTrack Scenarios",
    width=600,
    height=80
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0).configure_title(fontSize=18)

alt.Chart(...)